In [1]:
import pandas as pd
import datetime as dt

## GET STOPS SEQUENCE ALONG EACH LINE

In [3]:
stop_times = pd.read_csv('../GTFS_nyc_Subway/stop_times.txt')

In [40]:
stop_times['service_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[0])
stop_times['sub_trip_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[1])
stop_times['train+direction']=stop_times['trip_id'].apply(lambda x: x.split('_')[2])
del stop_times['stop_headsign']
del stop_times['pickup_type']
del stop_times['drop_off_type']
del stop_times['shape_dist_traveled']
stop_times['train'] = stop_times['train+direction'].apply(lambda x: x.split('.')[0])
stop_times['day'] = stop_times['service_id'].apply(lambda x: x[-3:])

In [107]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
0,A20161106SAT_036000_GS.N01R,06:00:00,06:00:00,901N,1,A20161106SAT,036000,GS.N01R,GS,SAT
1,A20161106SAT_036000_GS.N01R,06:01:30,06:01:30,902N,2,A20161106SAT,036000,GS.N01R,GS,SAT
2,A20161106SAT_036400_GS.S01R,06:04:00,06:04:00,902S,1,A20161106SAT,036400,GS.S01R,GS,SAT
3,A20161106SAT_036400_GS.S01R,06:05:30,06:05:30,901S,2,A20161106SAT,036400,GS.S01R,GS,SAT
4,A20161106SAT_037000_GS.N01R,06:10:00,06:10:00,901N,1,A20161106SAT,037000,GS.N01R,GS,SAT


In [155]:
def find_the_longest(line):
    """
    Find the longest route of a certain line,
    return how many stops the longest routes have,
    and route id, service id
    """
    
    North = filter( lambda x: x[3]=='N', set(stop_times['train+direction']))
    South = filter( lambda x: x[3]=='S', set(stop_times['train+direction']))
    ### some train longest route of south bound and north bound are different, 
    ### eg. A train, North bound max 59 stops, South bound max 58 stops.
    
    ## North bound:
    N_length=stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction'].isin(North))]['stop_sequence'].max()
    N_train_direction = list(set(stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction'].isin(North))&
                       (stop_times['stop_sequence']==N_length)]['train+direction']))[0]
    ## South bound:
    S_length=stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction'].isin(South))]['stop_sequence'].max()
    S_train_direction = list(set(stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction'].isin(South))&
                       (stop_times['stop_sequence']==S_length)]['train+direction']))[0]
    
    service_id = list(set(stop_times[(stop_times['train']==line)&
                      (stop_times['day']=='WKD')&
                      (stop_times['stop_sequence']==N_length)]['service_id']))[0]
    return N_length, N_train_direction, S_length, S_train_direction, service_id

In [156]:
train={}
for i in set(stop_times['train']):
    train[i] = find_the_longest(i)
train

### H is S train in JFK,
### GS is Time Sq-42 St to Grand Central,
### FS is near brooklyn prospect park

{'1': (38, '1..N02R', 38, '1..S02R', 'A20161106WKD'),
 '2': (61, '2..N08R', 61, '2..S08R', 'A20161106WKD'),
 '3': (34, '3..N01R', 34, '3..S01R', 'A20161106WKD'),
 '4': (54, '4..N13R', 54, '4..S13R', 'A20161106WKD'),
 '5': (39, '5..N60R', 36, '5..S03R', 'A20161106WKD'),
 '6': (38, '6..N01R', 38, '6..S01R', 'A20161106WKD'),
 '7': (22, '7..N97R', 22, '7..S97R', 'A20161106WKD'),
 'A': (59, 'A..N09R', 58, 'A..S74R', 'B20161106WKD'),
 'B': (37, 'B..N45R', 37, 'B..S45R', 'B20161106WKD'),
 'C': (40, 'C..N04R', 40, 'C..S04R', 'B20161106WKD'),
 'D': (41, 'D..N05R', 41, 'D..S05R', 'B20161106WKD'),
 'E': (32, 'E..N05R', 32, 'E..S04R', 'B20161106WKD'),
 'F': (45, 'F..N69R', 45, 'F..S69R', 'B20161106WKD'),
 'FS': (4, 'FS.N01R', 4, 'FS.S01R', 'B20161106WKD'),
 'G': (21, 'G..N14R', 21, 'G..S14R', 'B20161106WKD'),
 'GS': (2, 'GS.N01R', 2, 'GS.S03R', 'A20161106WKD'),
 'H': (5, 'H..N21R', 5, 'H..S21R', 'B20161106WKD'),
 'J': (30, 'J..N12R', 30, 'J..S12R', 'B20161106WKD'),
 'L': (24, 'L..N01R', 24, 'L..S0

In [197]:
df_1= pd.DataFrame(columns=['train','bound','stop_sequence','stop_id',
                            'service_id','route_id'])
for i in train.keys():
#for i in ['1']:
    stops_1 = train[i][0]
    direction_1= train[i][1]
    stops_2 = train[i][2]
    direction_2= train[i][3]
    service_id = train[i][4]
    
    df_2= stop_times[(stop_times['train+direction'] == direction_1)&
            (stop_times['train']==i)&
            (stop_times['service_id']==service_id )].iloc[:stops_1][['stop_id','stop_sequence']]
    df_2['train'] = i
    df_2['bound'] = direction_1[3]
    df_2['service_id'] = service_id
    df_2['route_id'] = direction_1
    df_1 = pd.concat([df_1,df_2])
    
    
    df_2= stop_times[(stop_times['train+direction'] == direction_2)&
            (stop_times['train']==i)&
            (stop_times['service_id']==service_id )].iloc[:stops_2][['stop_id','stop_sequence']]
    df_2['train'] = i
    df_2['bound'] = direction_2[3]
    df_2['service_id'] = service_id
    df_2['route_id'] = direction_2
    df_1 = pd.concat([df_1,df_2])
    

In [200]:
df_1 = df_1.sort_values(['train','bound','stop_sequence'])
df_1 = df_1.reset_index(drop= True)
stops = pd.read_csv('../GTFS_nyc_Subway/stops.txt')
df_1 = df_1.merge(stops[['stop_id','stop_name','stop_lat','stop_lon']],on=['stop_id'],how='left')

In [241]:
df_1.to_csv('../cleaned_data/subway_stops_sequence_weekday1(longest_route)')

In [246]:
def find_the_common(line):
    """
    Find the most common route of a certain line,
    return how many stops the longest routes have,
    and route id, service id
    """
    
    keys = stop_times[(stop_times['day']=='WKD')&
           (stop_times['train']==line)&
           (stop_times['stop_sequence']==1)]['train+direction'].value_counts().keys()
    
    North = filter( lambda x: x[3]=='N', keys)[0]  ## route id
    South = filter( lambda x: x[3]=='S', keys)[0]
    
    ### some train longest route of south bound and north bound are different, 
    ### eg. A train, North bound max 59 stops, South bound max 58 stops.
    
    ## North bound:
    N_length=stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction']==North)]['stop_sequence'].max()
#     N_train_direction = list(set(stop_times[(stop_times['train']==line)&
#                        (stop_times['day']=='WKD')&
#                        (stop_times['train+direction']==North)&
#                        (stop_times['stop_sequence']==N_length)]['train+direction']))[0]
    ## South bound:
    S_length=stop_times[(stop_times['train']==line)&
                       (stop_times['day']=='WKD')&
                       (stop_times['train+direction']==South)]['stop_sequence'].max()
#     S_train_direction = list(set(stop_times[(stop_times['train']==line)&
#                        (stop_times['day']=='WKD')&
#                        (stop_times['train+direction']==South)&
#                        (stop_times['stop_sequence']==S_length)]['train+direction']))[0]
    
    service_id = list(set(stop_times[(stop_times['train']==line)&
                      (stop_times['day']=='WKD')&
                      (stop_times['stop_sequence']==N_length)]['service_id']))[0]
    return N_length, North, S_length, South, service_id

In [266]:
train2={}
for i in set(stop_times['train']):
    train2[i] = find_the_common(i)
train2

### H is S train in JFK,
### GS is Time Sq-42 St to Grand Central,
### FS is near brooklyn prospect park

{'1': (38, '1..N02R', 38, '1..S02R', 'A20161106WKD'),
 '2': (49, '2..N01R', 49, '2..S01R', 'A20161106WKD'),
 '3': (34, '3..N01R', 34, '3..S01R', 'A20161106WKD'),
 '4': (28, '4..N06R', 28, '4..S06R', 'A20161106WKD'),
 '5': (36, '5..N66R', 36, '5..S03R', 'A20161106WKD'),
 '6': (38, '6..N01R', 38, '6..S01R', 'A20161106WKD'),
 '7': (22, '7..N97R', 22, '7..S97R', 'A20161106WKD'),
 'A': (30, 'A..N54R', 30, 'A..S54X010', 'B20161106WKD'),
 'B': (27, 'B..N46R', 27, 'B..S46R', 'B20161106WKD'),
 'C': (40, 'C..N04R', 40, 'C..S04R', 'B20161106WKD'),
 'D': (36, 'D..N07R', 36, 'D..S07R', 'B20161106WKD'),
 'E': (20, 'E..N66R', 20, 'E..S71R', 'B20161106WKD'),
 'F': (45, 'F..N69R', 45, 'F..S69R', 'B20161106WKD'),
 'FS': (4, 'FS.N01R', 4, 'FS.S01R', 'B20161106WKD'),
 'G': (21, 'G..N14R', 21, 'G..S14R', 'B20161106WKD'),
 'GS': (2, 'GS.N03R', 2, 'GS.S01R', 'A20161106WKD'),
 'H': (5, 'H..N21R', 5, 'H..S21R', 'B20161106WKD'),
 'J': (30, 'J..N12R', 30, 'J..S12R', 'B20161106WKD'),
 'L': (24, 'L..N01R', 24, 'L.

In [267]:
df_1= pd.DataFrame(columns=['train','bound','stop_sequence','stop_id',
                            'service_id','route_id'])
for i in train2.keys():
#for i in ['1']:
    stops_1 = train2[i][0]
    direction_1= train2[i][1]
    stops_2 = train2[i][2]
    direction_2= train2[i][3]
    service_id = train2[i][4]
    
    df_2= stop_times[(stop_times['train+direction'] == direction_1)&
            (stop_times['train']==i)&
            (stop_times['service_id']==service_id )].iloc[:stops_1][['stop_id','stop_sequence']]
    df_2['train'] = i
    df_2['bound'] = direction_1[3]
    df_2['service_id'] = service_id
    df_2['route_id'] = direction_1
    df_1 = pd.concat([df_1,df_2])
    
    
    df_2= stop_times[(stop_times['train+direction'] == direction_2)&
            (stop_times['train']==i)&
            (stop_times['service_id']==service_id )].iloc[:stops_2][['stop_id','stop_sequence']]
    df_2['train'] = i
    df_2['bound'] = direction_2[3]
    df_2['service_id'] = service_id
    df_2['route_id'] = direction_2
    df_1 = pd.concat([df_1,df_2])
    

In [269]:
df_1 = df_1.sort_values(['train','bound','stop_sequence'])
df_1 = df_1.reset_index(drop= True)
stops = pd.read_csv('../GTFS_nyc_Subway/stops.txt')
df_1 = df_1.merge(stops[['stop_id','stop_name','stop_lat','stop_lon']],on=['stop_id'],how='left')

In [255]:
df_1.head()

,bound,route_id,service_id,stop_id,stop_sequence,train,stop_name,stop_lat,stop_lon
0,N,1..N02R,A20161106WKD,140N,1,1,South Ferry Loop,40.701411,-74.013205
1,N,1..N02R,A20161106WKD,139N,2,1,Rector St,40.707513,-74.013783
2,N,1..N02R,A20161106WKD,138N,3,1,Cortlandt St,40.711835,-74.012188
3,N,1..N02R,A20161106WKD,137N,4,1,Chambers St,40.715478,-74.009266
4,N,1..N02R,A20161106WKD,136N,5,1,Franklin St,40.719318,-74.006886


In [270]:
### in most common route ,D line doesn't stop at Dekalb Av, but in longest route, it will
df_1[df_1['train']=='D'].loc[698:700]

,bound,route_id,service_id,stop_id,stop_sequence,train,stop_name,stop_lat,stop_lon
698,N,D..N07R,B20161106WKD,R31N,15,D,Atlantic Av - Barclays Ctr,40.683666,-73.978810
699,N,D..N07R,B20161106WKD,D22N,16,D,Grand St,40.718267,-73.993753
700,N,D..N07R,B20161106WKD,D21N,17,D,Broadway-Lafayette St,40.725297,-73.996204


In [257]:
df_1.to_csv('../cleaned_data/subway_stops_sequence_weekday2(most_common_route)')

## GET DURATION BETWEEN STOPS

In [271]:
train2

{'1': (38, '1..N02R', 38, '1..S02R', 'A20161106WKD'),
 '2': (49, '2..N01R', 49, '2..S01R', 'A20161106WKD'),
 '3': (34, '3..N01R', 34, '3..S01R', 'A20161106WKD'),
 '4': (28, '4..N06R', 28, '4..S06R', 'A20161106WKD'),
 '5': (36, '5..N66R', 36, '5..S03R', 'A20161106WKD'),
 '6': (38, '6..N01R', 38, '6..S01R', 'A20161106WKD'),
 '7': (22, '7..N97R', 22, '7..S97R', 'A20161106WKD'),
 'A': (30, 'A..N54R', 30, 'A..S54X010', 'B20161106WKD'),
 'B': (27, 'B..N46R', 27, 'B..S46R', 'B20161106WKD'),
 'C': (40, 'C..N04R', 40, 'C..S04R', 'B20161106WKD'),
 'D': (36, 'D..N07R', 36, 'D..S07R', 'B20161106WKD'),
 'E': (20, 'E..N66R', 20, 'E..S71R', 'B20161106WKD'),
 'F': (45, 'F..N69R', 45, 'F..S69R', 'B20161106WKD'),
 'FS': (4, 'FS.N01R', 4, 'FS.S01R', 'B20161106WKD'),
 'G': (21, 'G..N14R', 21, 'G..S14R', 'B20161106WKD'),
 'GS': (2, 'GS.N03R', 2, 'GS.S01R', 'A20161106WKD'),
 'H': (5, 'H..N21R', 5, 'H..S21R', 'B20161106WKD'),
 'J': (30, 'J..N12R', 30, 'J..S12R', 'B20161106WKD'),
 'L': (24, 'L..N01R', 24, 'L.

In [341]:
duration=pd.DataFrame(columns=['train','service_id','route_id','bound',
                               'from_stop_id','from_stop','from_stop_sequence',
                               'to_stop_id','to_stop','to_stop_sequence','duration'])

for i in train2.keys():
#for i in ['1']:
    stops_1 = train2[i][0]       #length of north bound route
    route_id_1= train2[i][1]    # route id for north bound
    stops_2 = train2[i][2]       #length of north bound route
    route_id_2= train2[i][3]    # ruote id for north bound
    service_id = train2[i][4]    # service id for weekday
    
    ### north bound
    df= stop_times[(stop_times['train+direction']==route_id_1)].iloc[:stops_1]
    df.arrival_time = pd.to_datetime(df.arrival_time)

    duration_li=[]
    for j in df.index[1:]:
        duration_li.append(int((df.loc[j,'arrival_time']-
                                df.loc[j-1,'arrival_time']).total_seconds()))
    duration_1 = pd.DataFrame(data=duration_li,columns=['duration'])
    duration_1['route_id'] = route_id_1
    duration_1['service_id'] =service_id
    duration_1['bound'] = 'N'
    duration_1['train'] = i
    duration_1['from_stop_sequence']= range(1,stops_1)
    duration_1['to_stop_sequence']= range(2,stops_1+1)
    duration_1['from_stop_id'] = df['stop_id'][:-1].values
    duration_1['to_stop_id'] = df['stop_id'][1:].values
    duration = pd.concat([duration,duration_1])
    
    ### south bound
    df= stop_times[(stop_times['train+direction']==route_id_2)].iloc[:stops_2]
    df.arrival_time = pd.to_datetime(df.arrival_time)

    duration_li=[]
    for j in df.index[1:]:
        duration_li.append(int((df.loc[j,'arrival_time']-
                                df.loc[j-1,'arrival_time']).total_seconds()))
    duration_1 = pd.DataFrame(data=duration_li,columns=['duration'])
    duration_1['route_id'] = route_id_2
    duration_1['service_id'] =service_id
    duration_1['bound'] = 'S'
    duration_1['train'] = i
    duration_1['from_stop_sequence']= range(1,stops_2)
    duration_1['to_stop_sequence']= range(2,stops_2+1)
    duration_1['from_stop_id'] = df['stop_id'][:-1].values
    duration_1['to_stop_id'] = df['stop_id'][1:].values
    duration = pd.concat([duration,duration_1])

In [346]:
stops.head(2)

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,101,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,1,NaN
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101


In [373]:
stops = pd.read_csv('../GTFS_nyc_Subway/stops.txt')
duration= duration.reset_index(drop= True)
for i in range(len(duration)):
    from_stop_id = duration.loc[i,'from_stop_id']
    to_stop_id = duration.loc[i,'to_stop_id']
    duration.loc[i,'from_stop'] = stops[stops['stop_id']==from_stop_id]['stop_name'].values[0]
    duration.loc[i,'to_stop'] = stops[stops['stop_id']==to_stop_id]['stop_name'].values[0]

In [375]:
duration.to_csv('../cleaned_data/subway_duration_between_stops_weekday(common_route)')

## shapes.txt, is showing the sequence of polyline, if mapping the trace of any train route

In [9]:
shapes = pd.read_csv('../GTFS_nyc_Subway/shapes.txt')
shapes.head()

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,1..N03R,40.702068,-74.013664,0,NaN
1,1..N03R,40.703199,-74.014792,1,NaN
2,1..N03R,40.703226,-74.014820,2,NaN
3,1..N03R,40.703253,-74.014846,3,NaN
4,1..N03R,40.703280,-74.014870,4,NaN


In [26]:
### the route of S train from Time Sq - 42 St to Grand Central
shapes[shapes.shape_id == 'GS.N01R']

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
88597,GS.N01R,40.752769,-73.979189,0,NaN
88598,GS.N01R,40.755522,-73.985728,1,NaN
88599,GS.N01R,40.755545,-73.985777,2,NaN
88600,GS.N01R,40.755570,-73.985823,3,NaN
88601,GS.N01R,40.755595,-73.985867,4,NaN
88602,GS.N01R,40.755622,-73.985909,5,NaN
88603,GS.N01R,40.755649,-73.985949,6,NaN
88604,GS.N01R,40.755678,-73.985987,7,NaN
88605,GS.N01R,40.755708,-73.986022,8,NaN
88606,GS.N01R,40.755738,-73.986055,9,NaN
